In [1]:
import sys
sys.path.append("../..")
from thesis.utils.utils import setup_ray, save, load, Experiment
path = "../.."
setup_ray(path = path, unidirectional = False, seed=69)

2023-06-04 09:40:56,530	INFO worker.py:1528 -- Started a local Ray instance.


In [2]:
env_args = dict(
    fleetsize = 8,
    fleetsize_upper = 20,
    max_fleetsize = 20,    
    pseudo_routing = False,
    pseudo_dispatcher = False,
    pseudo_dispatcher_clever = False,
    #pseudo_dispatcher_distance = 0.3,
    routing_agent_death= True,
    death_on_target = False,
    transform_dispatching_partobs = True,
    direction_reward = -0.1,
    sim_config = dict(
        dispatch = True,
        routing_ma = True,
        dispatching_ma = True,
        reward_reached_target = 10,
        #reward_reached_target_by_time = True, 
        reward_wrong_target = -1,
        reward_removed_for_block = -20, 
        reward_target_distance = 0,
        reward_invalid= -0.1,
        reward_duration = -0.5,
        reward_pass = 0.2,
        block_timeout = 60,
        station_separate = False,
        reward_accepted_in_station = 0.5,
        reward_declined_in_station = -0.25,
        #reward_part_completed = 5,
        reward_geo_operation=0.25,
        #reward_rework_operation=1,
        #reward_respot_operation=1,
        reward_reduce = -0.005,
        #reward_balance = -5,
        routing_interval = 2,
        dispatching_interval=360,
        io_quote = 0.9  ,
        availability = 0.9,
        mttr = 5,
        fixed_fleets = True,
        remove_all_blocked = False,
    )
)

In [3]:
agv_model = dict(
    model = dict(
        custom_model = "gnn_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=32,
            with_action_mask=False,
            with_agvs=True,
            with_stations = False,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
            env_type = "matrix",
            n_convolutions = 2,
            #with_node_info = True,
        )
    )
)
dispatcher_model = dict(
    model = dict(
        custom_model = "lin_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=32,
            with_action_mask=True,
            with_agvs=True,
            with_stations = True,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
        )
    )
)

In [4]:
exp = Experiment("matrix_together")
for seed in [43]:
    exp.experiment(
        path = path,
        env_args = env_args, 
        agv_model = agv_model,
        dispatcher_model=dispatcher_model,
        run_name="LAST", 
        env = "matrix",
        algo = "double",
        n_intervals =0,
        backup_interval=50,
        batch_size=128, 
        seed = seed,
        algo_params = {"gamma":0.9, "exploration_config":{"warmup_timesteps": 0,"epsilon_timesteps": 50000,"final_epsilon": 0.05,"initial_epsilon": 0.05,"type": "EpsilonGreedy"}},
        lr = 1e-4,
        two_fleets = True,
        #train_agv = False,
        load_agv="../../models/matrix_together/LAST_8_20_2023-06-03_01-06-00/checkpoint_000400"
    )
    #load(exp.trainer, "agv", "../../models/final")
    #load(exp.trainer, "dispatcher1", "../../models/fin_disp_1_red")
    #load(exp.trainer, "dispatcher2", "../../models/fin_disp_2_red")
    exp.keep_training(2)

2023-06-04 09:41:09,440	INFO algorithm.py:457 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_40359_eo56yd28)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
2023-06-04 09:41:31,033	INFO trainable.py:164 -- Trainable.setup took 21.598 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-06-04 09:41:31,415	INFO trainable.py:766 -- Restored on 127.0.0.1 from checkpoint: ..\..\models\matrix_together\LAST_8_20_2023-06-03_01-06-00\checkpoint_000400
2023-06-04 09:41:31,415	INFO trainable.py:775 -- Current state after restoring: {'_iteration': 400, '_timesteps_total': None, '_time_total': 35191.72051215172, '_episodes_total': 346}


In [5]:
#exp.trainer.save("../../models/TEST")

In [6]:
#exp.keep_training(4)

In [7]:
exp.keep_training(2)

In [8]:
exp.keep_training(2)